In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import json
import re
pd.set_option('max_colwidth', 400)

### Extract the crowdfunding.xlsx Data

In [2]:
# Read the data into a Pandas DataFrame
crowdfunding_info_df = pd.read_excel('Resources/crowdfunding.xlsx')
crowdfunding_info_df.head(2)

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock


In [3]:
# Get a brief summary of the crowdfunding_info DataFrame.
crowdfunding_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   cf_id                    1000 non-null   int64 
 1   contact_id               1000 non-null   int64 
 2   company_name             1000 non-null   object
 3   blurb                    1000 non-null   object
 4   goal                     1000 non-null   int64 
 5   pledged                  1000 non-null   int64 
 6   outcome                  1000 non-null   object
 7   backers_count            1000 non-null   int64 
 8   country                  1000 non-null   object
 9   currency                 1000 non-null   object
 10  launched_at              1000 non-null   int64 
 11  deadline                 1000 non-null   int64 
 12  staff_pick               1000 non-null   bool  
 13  spotlight                1000 non-null   bool  
 14  category & sub-category  1000 non-null   

### Create the Category and Subcategory DataFrames
---
**Create a Category DataFrame that has the following columns:**
- A "category_id" column that is numbered sequential form 1 to the length of the number of unique categories.
- A "category" column that has only the categories.

Export the DataFrame as a `category.csv` CSV file.

**Create a SubCategory DataFrame that has the following columns:**
- A "subcategory_id" column that is numbered sequential form 1 to the length of the number of unique subcategories.
- A "subcategory" column that has only the subcategories. 

Export the DataFrame as a `subcategory.csv` CSV file.

In [4]:
# split the cat/subcat series
parsed_cats = crowdfunding_info_df['category & sub-category'].str.split("/",expand=True)

#rename those split cols to cat and subcat respectively
parsed_cats = parsed_cats.rename(columns={0:'category',1:'subcategory'})

#rejoin those to the main df
crowdfunding_info_df = crowdfunding_info_df.join(parsed_cats)

#view df
crowdfunding_info_df.head(2)

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock


In [5]:
# Get the crowdfunding_info_df columns.
crowdfunding_info_df.columns


Index(['cf_id', 'contact_id', 'company_name', 'blurb', 'goal', 'pledged',
       'outcome', 'backers_count', 'country', 'currency', 'launched_at',
       'deadline', 'staff_pick', 'spotlight', 'category & sub-category',
       'category', 'subcategory'],
      dtype='object')

In [6]:
# Get the unique categories and subcategories in separate lists.
categories = list(crowdfunding_info_df['category'].unique())
subcategories = list(crowdfunding_info_df['subcategory'].unique())

print(categories)
print()
print(subcategories)

['food', 'music', 'technology', 'theater', 'film & video', 'publishing', 'games', 'photography', 'journalism']

['food trucks', 'rock', 'web', 'plays', 'documentary', 'electric music', 'drama', 'indie rock', 'wearables', 'nonfiction', 'animation', 'video games', 'shorts', 'fiction', 'photography books', 'radio & podcasts', 'metal', 'jazz', 'translations', 'television', 'mobile games', 'world music', 'science fiction', 'audio']


In [7]:
# Get the number of distinct values in the categories and subcategories lists.
print(len(categories))
print(len(subcategories))

9
24


In [8]:
# Use a list comprehension to add "cat" to each category_id. 
cat_ids = [f'cat{ind}' for ind in np.arange(1, len(categories)+1)]

# Use a list comprehension to add "subcat" to each subcategory_id.    
subcat_ids = [f'subcat{ind}' for ind in np.arange(1, len(subcategories)+1)]
    
print(cat_ids)
print()
print(subcat_ids)

['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']

['subcat1', 'subcat2', 'subcat3', 'subcat4', 'subcat5', 'subcat6', 'subcat7', 'subcat8', 'subcat9', 'subcat10', 'subcat11', 'subcat12', 'subcat13', 'subcat14', 'subcat15', 'subcat16', 'subcat17', 'subcat18', 'subcat19', 'subcat20', 'subcat21', 'subcat22', 'subcat23', 'subcat24']


In [9]:
# Create a category DataFrame with the category_id array as the category_id and categories list as the category name.
cat_ids = pd.DataFrame(cat_ids)
cat_ids = cat_ids.rename(columns={0:'category_id'})

categories = pd.DataFrame(categories)
categories = categories.rename(columns={0:'categories'})

category_df = categories.join(cat_ids)

category_df = category_df[['category_id','categories']]

# Create a category DataFrame with the subcategory_id array as the subcategory_id and subcategories list as the subcategory name. 
subcat_ids = pd.DataFrame(subcat_ids)
subcat_ids = subcat_ids.rename(columns={0:'subcategory_id'})

subcategories = pd.DataFrame(subcategories)
subcategories = subcategories.rename(columns={0:'subcategories'})

subcategory_df = subcategories.join(subcat_ids)

subcategory_df = subcategory_df[['subcategory_id','subcategories']]


In [10]:
category_df.columns

Index(['category_id', 'categories'], dtype='object')

In [11]:
subcategory_df.head(2)

,subcategory_id,subcategories
0,subcat1,food trucks
1,subcat2,rock


In [12]:
# Export categories_df and subcategories_df as CSV files.
category_df.to_csv("Resources/category.csv", index=False)

subcategory_df.to_csv("Resources/subcategory.csv", index=False)

### Campaign DataFrame
----
**Create a Campaign DataFrame that has the following columns:**
- The "cf_id" column.
- The "contact_id" column.
- The “company_name” column.
- The "blurb" column is renamed as "description."
- The "goal" column.
- The "goal" column is converted to a `float` datatype.
- The "pledged" column is converted to a `float` datatype. 
- The "backers_count" column. 
- The "country" column.
- The "currency" column.
- The "launched_at" column is renamed as "launch_date" and converted to a datetime format. 
- The "deadline" column is renamed as "end_date" and converted to a datetime format.
- The "category_id" with the unique number matching the “category_id” from the category DataFrame. 
- The "subcategory_id" with the unique number matching the “subcategory_id” from the subcategory DataFrame.
- And, create a column that contains the unique four-digit contact ID number from the `contact.xlsx` file.
 

Then export the DataFrame as a `campaign.csv` CSV file.


In [13]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df. 
campaign_df = crowdfunding_info_df.copy()
campaign_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


In [14]:
# Rename the blurb, launched_at, and deadline columns.
campaign_df = campaign_df.rename(
    columns={
        'blurb':'description',
        'launched_at':'launched_date',
        'deadline':'end_date'
    }
)

campaign_df.head(2)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock


In [15]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df = campaign_df.astype(
    {
        'goal':'float64',
        'pledged':'float64'
    }
)
campaign_df.head(2)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock


In [28]:
# Check the datatypes
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cf_id                    1000 non-null   int64  
 1   contact_id               1000 non-null   int64  
 2   company_name             1000 non-null   object 
 3   description              1000 non-null   object 
 4   goal                     1000 non-null   float64
 5   pledged                  1000 non-null   float64
 6   outcome                  1000 non-null   object 
 7   backers_count            1000 non-null   int64  
 8   country                  1000 non-null   object 
 9   currency                 1000 non-null   object 
 10  launched_date            1000 non-null   object 
 11  end_date                 1000 non-null   object 
 12  staff_pick               1000 non-null   bool   
 13  spotlight                1000 non-null   bool   
 14  category & sub-category  

In [17]:
#convert launch/end dates to datetime
from datetime import datetime as dt

campaign_df["launched_date"] = pd.to_datetime(campaign_df["launched_date"], unit='s').dt.strftime('%Y-%m-%d') 
campaign_df["end_date"] = pd.to_datetime(campaign_df["end_date"], unit='s').dt.strftime('%Y-%m-%d')
campaign_df.head(2)


,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13,2021-03-01,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25,2021-05-25,False,True,music/rock,music,rock


In [18]:
# Merge the campaign_df with the category_df on the "category" column and 
# the subcategory_df on the "subcategory" column.

campaign_merged_df = pd.merge(campaign_df, category_df,
                 left_on='category',
                 right_on='categories')

campaign_merged_df = pd.merge(campaign_merged_df, subcategory_df,
                 left_on='subcategory',
                 right_on='subcategories')

campaign_merged_df.head(2)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,...,end_date,staff_pick,spotlight,category & sub-category,category,subcategory,category_id,categories,subcategory_id,subcategories
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,...,2021-03-01,False,False,food/food trucks,food,food trucks,cat1,food,subcat1,food trucks
1,1175,2288,Werner-Bryant,Virtual uniform frame,1800.0,7991.0,successful,222,US,USD,...,2021-01-30,False,False,food/food trucks,food,food trucks,cat1,food,subcat1,food trucks


In [19]:
# Drop unwanted columns
campaign_merged_df=campaign_merged_df.drop(columns=['categories','subcategories'])
campaign_merged_df.head(2)


,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory,category_id,subcategory_id
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13,2021-03-01,False,False,food/food trucks,food,food trucks,cat1,subcat1
1,1175,2288,Werner-Bryant,Virtual uniform frame,1800.0,7991.0,successful,222,US,USD,2020-06-20,2021-01-30,False,False,food/food trucks,food,food trucks,cat1,subcat1


In [29]:
# Export the DataFrame as a CSV file. 
campaign_merged_df.to_csv("Resources/campaign.csv", index=False)

### Extract the contacts.xlsx Data.

In [21]:
# Read the data into a Pandas DataFrame. Use the `header=2` parameter when reading in the data.
contact_info_df = pd.read_excel('Resources/contacts.xlsx', header=2)
contact_info_df.head()


,
0,contact_info
1,"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}"
2,"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}"
3,"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}"
4,"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}"


### Create the Contacts DataFrame 
---
**Create a Contacts DataFrame that has the following columns:**
- A column named "contact_id"  that contains the unique number of the contact person.
- A column named "first_name" that contains the first name of the contact person.
- A column named "last_name" that contains the first name of the contact person.
- A column named "email" that contains the email address of the contact person

Then export the DataFrame as a `contacts.csv` CSV file.

### Option 1: Use Pandas to create the contacts DataFrame.

In [22]:
# Iterate through the contact_info_df and convert each row to a dictionary.
df = []

for i in range(1,len(contact_info_df)):
    contact = contact_info_df.loc[i][0]
    contact = json.loads(contact)
    df.append(contact)

# Create a contact_info DataFrame and add each list of values, i.e., each row 
# to the 'contact_id', 'name', 'email' columns.
contacts_df = pd.DataFrame.from_dict(df)
contacts_df


,contact_id,name,email
0,4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana Ellis,mariana.ellis@rossi.org
2,4187,Sofie Woods,sofie.woods@riviere.com
3,4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel Sorgatz,samuel.sorgatz@gmail.com
...,...,...,...
995,3684,Whitney Noack,whitney.noack@laboratorios.org
996,5784,Gelsomina Migliaccio,gelsomina.migliaccio@junk.com
997,1498,Evangelista Pereira,evangelista.pereira@thompson-peterson.biz
998,6073,Gareth Comolli,gareth.comolli@tiscali.fr


In [23]:
# Check the datatypes.
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   name        1000 non-null   object
 2   email       1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [24]:
# Create a "first"name" and "last_name" column with the first and last names from the "name" column. 
names = contacts_df['name'].str.split(" ",expand=True)
names.columns = ['first_name','last_name']

contacts_df = contacts_df.join(names)

# Drop the contact_name column
contacts_df = contacts_df.drop(columns = ['name'])
contacts_df


,contact_id,email,first_name,last_name
0,4661,cecilia.velasco@rodrigues.fr,Cecilia,Velasco
1,3765,mariana.ellis@rossi.org,Mariana,Ellis
2,4187,sofie.woods@riviere.com,Sofie,Woods
3,4941,jeanette.iannotti@yahoo.com,Jeanette,Iannotti
4,2199,samuel.sorgatz@gmail.com,Samuel,Sorgatz
...,...,...,...,...
995,3684,whitney.noack@laboratorios.org,Whitney,Noack
996,5784,gelsomina.migliaccio@junk.com,Gelsomina,Migliaccio
997,1498,evangelista.pereira@thompson-peterson.biz,Evangelista,Pereira
998,6073,gareth.comolli@tiscali.fr,Gareth,Comolli


In [25]:
# Reorder the columns
contacts_df = contacts_df[['contact_id', 'first_name', 'last_name', 'email']]
contacts_df

,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org
2,4187,Sofie,Woods,sofie.woods@riviere.com
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com
...,...,...,...,...
995,3684,Whitney,Noack,whitney.noack@laboratorios.org
996,5784,Gelsomina,Migliaccio,gelsomina.migliaccio@junk.com
997,1498,Evangelista,Pereira,evangelista.pereira@thompson-peterson.biz
998,6073,Gareth,Comolli,gareth.comolli@tiscali.fr


In [26]:
# Check the datatypes one more time before exporting as CSV file.
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [27]:
# Export the DataFrame as a CSV file. 
contacts_df.to_csv("Resources/contacts.csv", encoding='utf8', index=False)
